In [1]:
import sys
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# 擷取此新聞的發佈日期
def get_date(news_block_node):
    date_string = news_block_node.find(class_="news_date").string.split('|')[0][2:-1]
    return(datetime.strptime(date_string, '%Y.%m.%d').strftime('%Y-%m-%d'))

# 擷取此新聞的標題
def get_title(news_block_node):
    return news_block_node.find(class_="news_title").a.string

# 擷取此新聞連結的網址
def get_link(news_block_node):
    return news_block_node.find(class_="news_title").a.get('href')

# 進入新聞連結的頁面，擷取裡面的文字
def get_content(link):
    r = requests.get(link)
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    article_node = soup.find(itemprop='articleBody')
    article = article_node.get_text()
    return article.replace("\n", "")

In [3]:
def get_news_info(each_news):
    date  = get_date(each_news)    # 擷取此新聞的發佈日期
    title = get_title(each_news)   # 擷取此新聞的標題
    link  = get_link(each_news)    # 擷取此新聞連結的網址
    content = get_content(link)    # 進入新聞連結的頁面，擷取裡面的文字
    
    info = {'date' : date,
            'title': title,
            'link' : link,
            'content': content}
    return(info)

def get_page_news(page_url):
    r = requests.get(page_url)     # 使用 GET 方式下載網頁
    r.encoding = "UTF-8"

    # r.text 是網頁的 HTML 原始碼
    soup = BeautifulSoup(r.text, 'html.parser')    # 以 Beautiful Soup 解析 HTML 程式碼
    news_blocks = soup.find_all(class_="news-list-item clearfix ")   # 找出網頁第一頁的所有新聞
    
    news = []
    for each_news in news_blocks:
        try:
            news_info = get_news_info(each_news)
#             print(get_title(each_news))
        except:
#             print('-------{}-------'.format())
            pass

        news.append(news_info)                    # 將每則新聞的資訊逐一加入至 list
    return(news)


# 只擷取此網站第 1 頁至第 270 頁的所有新聞
def get_new_talk_news(from_page=1, end_page=270, url="https://newtalk.tw/news/subcategory/2/政治/"):
    print("page_number from {} to {}".format(from_page, end_page -1))
    data = []
    for page_number in range(from_page, end_page):
        print("page_number: {}".format(page_number))
        data = data + get_page_news( url+str(page_number) )
    
    print('done')
    return(data)

In [4]:
data = get_new_talk_news(from_page=1, end_page=6)    # 只擷取此網站第 1 頁至第 270 頁的所有新聞

page_number from 1 to 5
page_number: 1
page_number: 2
page_number: 3
page_number: 4
page_number: 5
done


In [30]:
sys.setrecursionlimit(100000)   # 設定遞迴最多能到幾層   ps. 不知為啥要做此設定

# 將收集到的資訊保存至 data/new_talk.pkl
with open('data/new_talk.pkl', 'wb') as handle:
    pickle.dump(data, handle)